<h1>Оценка качества воздуха в регионах<h1>
<h2>Бинарная классификация<h2>
<h4>Файл с данными - air_quality.csv<h4>

In [47]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import classification_report

<h3>Константа<h3>

In [24]:
PATH = 'air_quality.csv'

<h3>Исследовательская часть<h3>

In [25]:
data = pd.read_csv(PATH)
data.head(10)

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Proximity_to_Industrial_Areas,Population_Density,air_quality
0,29.8,59.1,5.2,17.9,18.9,9.2,1.72,6.3,319,1
1,28.3,75.6,2.3,12.2,30.8,9.7,1.64,6.0,611,1
2,23.1,74.7,26.7,33.8,24.4,12.6,1.63,5.2,619,1
3,27.1,39.1,6.1,6.3,13.5,5.3,1.15,11.1,551,1
4,26.5,70.7,6.9,16.0,21.9,5.6,1.01,12.7,303,1
5,39.4,96.6,14.6,35.5,42.9,17.9,1.82,3.1,674,0
6,41.7,82.5,1.7,15.8,31.1,12.7,1.80,4.6,735,0
7,31.0,59.6,5.0,16.8,24.2,13.6,1.38,6.3,443,1
8,29.4,93.8,10.3,22.7,45.1,11.8,2.03,5.4,486,0
9,33.2,80.5,11.1,24.4,32.0,15.3,1.69,4.9,535,0


In [26]:
sizes = data.shape
empty = data.isnull().sum().sum()
print(f"Размерность данных: {sizes}")
print(f"Пропусков в данных: {empty}")
data.info()

Размерность данных: (5000, 10)
Пропусков в данных: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Temperature                    5000 non-null   float64
 1   Humidity                       5000 non-null   float64
 2   PM2.5                          5000 non-null   float64
 3   PM10                           5000 non-null   float64
 4   NO2                            5000 non-null   float64
 5   SO2                            5000 non-null   float64
 6   CO                             5000 non-null   float64
 7   Proximity_to_Industrial_Areas  5000 non-null   float64
 8   Population_Density             5000 non-null   int64  
 9   air_quality                    5000 non-null   int64  
dtypes: float64(8), int64(2)
memory usage: 390.8 KB


In [27]:
X = data.drop(columns=['air_quality'])
y = data['air_quality']

<h3>Разделение данных с учетом стратификации<h3>

In [28]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [29]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [30]:
class AirQualityDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets.values, dtype=torch.float32)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

<h3>Создание Dataset<h3>

In [31]:
train_dataset = AirQualityDataset(X_train, y_train)
val_dataset = AirQualityDataset(X_val, y_val)
test_dataset = AirQualityDataset(X_test, y_test)

<h3>Создание DataLoader<h3>

In [32]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [33]:
class AirQualityModel(nn.Module):
    def __init__(self, input_size):
        super(AirQualityModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

<h3>Инициализация модели<h3>

In [34]:
input_size = X_train.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AirQualityModel(input_size).to(device)


<h3>Функция потерь и оптимизатора<h3>

In [35]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [36]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    for features, targets in loader:
        features, targets = features.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(features).squeeze()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

def eval_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0
    all_targets, all_outputs = [], []
    with torch.no_grad():
        for features, targets in loader:
            features, targets = features.to(device), targets.to(device)
            outputs = model(features).squeeze()
            loss = criterion(outputs, targets)
            running_loss += loss.item()
            all_targets.extend(targets.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())
    return running_loss / len(loader), all_targets, all_outputs

In [38]:
n_epochs = 20
train_losses, val_losses = [], []

for epoch in range(n_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, _, _ = eval_epoch(model, val_loader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"Кол-во эпох {epoch+1}/{n_epochs}, Обучающая потеря: {train_loss:.4f}, Потеря валидации: {val_loss:.4f}")

Кол-во эпох 1/20, Обучающая потеря: 0.0514, Потеря валидации: 0.0691
Кол-во эпох 2/20, Обучающая потеря: 0.0497, Потеря валидации: 0.0698
Кол-во эпох 3/20, Обучающая потеря: 0.0483, Потеря валидации: 0.0735
Кол-во эпох 4/20, Обучающая потеря: 0.0481, Потеря валидации: 0.0715
Кол-во эпох 5/20, Обучающая потеря: 0.0472, Потеря валидации: 0.0689
Кол-во эпох 6/20, Обучающая потеря: 0.0452, Потеря валидации: 0.0719
Кол-во эпох 7/20, Обучающая потеря: 0.0444, Потеря валидации: 0.0688
Кол-во эпох 8/20, Обучающая потеря: 0.0433, Потеря валидации: 0.0690
Кол-во эпох 9/20, Обучающая потеря: 0.0432, Потеря валидации: 0.0704
Кол-во эпох 10/20, Обучающая потеря: 0.0431, Потеря валидации: 0.0700
Кол-во эпох 11/20, Обучающая потеря: 0.0415, Потеря валидации: 0.0702
Кол-во эпох 12/20, Обучающая потеря: 0.0399, Потеря валидации: 0.0735
Кол-во эпох 13/20, Обучающая потеря: 0.0395, Потеря валидации: 0.0755
Кол-во эпох 14/20, Обучающая потеря: 0.0385, Потеря валидации: 0.0699
Кол-во эпох 15/20, Обучающая 

<h3>Оценка на тестовой выборке<h3>

In [39]:
_, test_targets, test_outputs = eval_epoch(model, test_loader, criterion, device)
test_preds = [1 if p > 0.5 else 0 for p in test_outputs]


<h3>Отчет бинарной классификации<h3>

In [46]:
report = classification_report(test_targets, test_preds, target_names=["Низкий", "Высокий"])
print(report)

              precision    recall  f1-score   support

      Низкий       0.97      0.94      0.95       300
     Высокий       0.97      0.99      0.98       700

    accuracy                           0.97      1000
   macro avg       0.97      0.96      0.97      1000
weighted avg       0.97      0.97      0.97      1000

